In [1]:
from get_F1_data import getdata


In [15]:
import datetime
import yukinator
import pandas as pd
import numpy as np
import datetime
import re
import requests
YEAR = datetime.datetime.now().year
y = yukinator.Yuki()

def get_wikipedia_main_pic_url(article_title):
    endpoint = "https://en.wikipedia.org/w/api.php"

    params = {
        "action": "query",
        "format": "json",
        "titles": article_title,
        "prop": "pageimages",
        "pithumbsize": 300,  # Adjust the thumbnail size as needed
    }

    response = requests.get(endpoint, params=params)
    data = response.json()

    page_id = list(data["query"]["pages"].keys())[0]
    main_image_url = data["query"]["pages"][page_id]["thumbnail"]["source"]

    return main_image_url

def try_get_wikipedia_main_pic_url(x):
    try:
        result = get_wikipedia_main_pic_url(x)
    except Exception as e:
        # Handle the exception (e.g., print an error message)
        print(f"Error processing {x}: {e}")
        result = None  

    return result



def get_driver_info():
    drivers = pd.DataFrame([driver.to_flat_dict() for driver in y.get_drivers(year=YEAR)])
    # Define the regex pattern to match the input text
    pattern = r"http://en.wikipedia.org/wiki/"
    # Define the replacement text with the modified URL
    replacement = r""
    drivers['article_title'] = drivers['url'].apply(lambda x: re.sub(pattern, replacement, x))

    #manual modification of wiki article_titles
    conditions = [drivers["article_title"]=='Alexander_Albon', drivers["article_title"]=='Nico_H%C3%BClkenberg',drivers["article_title"]=='Sergio_P%C3%A9rez']
    values = ['Alex_Albon','Nico_Hülkenberg','Sergio_Pérez']
    drivers["article_title"] = np.select(conditions, values, default=drivers["article_title"])
    # url_dict = dict([(driver_code, (try_get_wikipedia_main_pic_url(driver_article_title))) for driver_code, driver_article_title in dict(zip(drivers["code"],drivers["article_title"])).items()])
    
    # url_df = pd.DataFrame(code = list(url_dict.keys(), pic_url = list(url_dict.values())))

    drivers['pic_url'] = drivers['article_title'].apply(lambda x: try_get_wikipedia_main_pic_url(x))

    return drivers



drivers = get_driver_info()







In [19]:
driver_and_team = df[["driverCode", "constructor"]].drop_duplicates()

In [31]:
driver_to_team = dict(zip(driver_and_team['driverCode'],driver_and_team['constructor']))

team_to_driver = dict()
for key, value in driver_to_team.items():
    if value in team_to_driver.keys():
        team_to_driver[value].append(key)
    else:
        team_to_driver[value] = [key]
team_to_driver

{'Williams': ['ALB', 'SAR'],
 'Aston Martin': ['ALO', 'STR'],
 'Alfa Romeo': ['BOT', 'ZHO'],
 'AlphaTauri': ['DEV', 'LAW', 'RIC', 'TSU'],
 'Alpine F1 Team': ['GAS', 'OCO'],
 'Mercedes': ['HAM', 'RUS'],
 'Haas F1 Team': ['HUL', 'MAG'],
 'Ferrari': ['LEC', 'SAI'],
 'McLaren': ['NOR', 'PIA'],
 'Red Bull': ['PER', 'VER']}

In [36]:
positions = df.pivot(index=['race'],columns="driverCode", values="positions").to_dict()
for key, value in positions.items():
    print(key)


ALB
ALO
BOT
DEV
GAS
HAM
HUL
LAW
LEC
MAG
NOR
OCO
PER
PIA
RIC
RUS
SAI
SAR
STR
TSU
VER
ZHO


In [12]:
list(set(df['constructor']))[0:5]





['Alfa Romeo', 'Alpine F1 Team', 'Haas F1 Team', 'AlphaTauri', 'McLaren']

In [38]:
drivers.set_index("code").to_dict(orient='index')




{'ALB': {'driverId': 'albon',
  'url': 'http://en.wikipedia.org/wiki/Alexander_Albon',
  'givenName': 'Alexander',
  'familyName': 'Albon',
  'dateOfBirth': datetime.date(1996, 3, 23),
  'nationality': 'Thai',
  'permanentNumber': 23,
  'article_title': 'Alex_Albon',
  'pic_url': 'https://upload.wikimedia.org/wikipedia/commons/thumb/1/1f/Alex_albon_%2851383514844%29_%28cropped%29.jpg/300px-Alex_albon_%2851383514844%29_%28cropped%29.jpg'},
 'ALO': {'driverId': 'alonso',
  'url': 'http://en.wikipedia.org/wiki/Fernando_Alonso',
  'givenName': 'Fernando',
  'familyName': 'Alonso',
  'dateOfBirth': datetime.date(1981, 7, 29),
  'nationality': 'Spanish',
  'permanentNumber': 14,
  'article_title': 'Fernando_Alonso',
  'pic_url': 'https://upload.wikimedia.org/wikipedia/commons/thumb/2/2b/Alonso_2016.jpg/300px-Alonso_2016.jpg'},
 'BOT': {'driverId': 'bottas',
  'url': 'http://en.wikipedia.org/wiki/Valtteri_Bottas',
  'givenName': 'Valtteri',
  'familyName': 'Bottas',
  'dateOfBirth': datetime.

In [46]:
def get_map_data(df, driver_info):
    """get favorite driver"""
    driver_and_team = df[["driverCode", "constructor"]].drop_duplicates()
    driver_to_team = dict(
        zip(driver_and_team["driverCode"], driver_and_team["constructor"])
    )
    team_to_driver = dict()
    for key, value in driver_to_team.items():
        if value in team_to_driver.keys():
            team_to_driver[value].append(key)
        else:
            team_to_driver[value] = [key]
    driver_info["contructor"] = driver_info["code"].apply(
        lambda x: driver_to_team[x]
    )
    return (team_to_driver, driver_info.set_index("code").to_dict(orient="index"))

team_to_driver, driver_info = get_map_data(df,drivers)


In [55]:
current_drivers = df[df['race'] == df['race'].max()].set_index("driverCode").to_dict(orient="index")

In [69]:
current_drivers

,race,driverCode,points,positions
constructor,,,,
Williams,17,ALB,23.0,13
Aston Martin,17,ALO,183.0,4
Alfa Romeo,17,BOT,10.0,14
AlphaTauri,17,DEV,0.0,20
Alpine F1 Team,17,GAS,46.0,11
Mercedes,17,HAM,194.0,3
Haas F1 Team,17,HUL,9.0,15
AlphaTauri,17,LAW,2.0,19
Ferrari,17,LEC,145.0,6


In [57]:
for key in current_drivers.keys():
    driver_info[key].update(current_drivers[key])

In [60]:
current_drivers = df[df['race'] == df['race'].max()].set_index("constructor").to_dict(orient="index")

In [62]:
driver_and_team = df[["driverCode", "constructor"]].drop_duplicates().set_index("driverCode").to_dict(orient="index")

In [85]:
df_team = df[["driverCode", "constructor"]].drop_duplicates()

driver_to_ctors = dict(zip(df_team["driverCode"],df_team["constructor"]))

ctors = df_team["constructor"].drop_duplicates()

colors = [
    '#0073C2FF', '#EFC000FF', '#868686FF', '#CD534CFF',
    '#7AA6DCFF', '#003C67FF', '#8F7700FF', '#3B3B3BFF',
    '#A73030FF', '#4A6990FF'
]

ctors_to_color =  dict(zip(ctors,colors))

driver_to_color = [(key, ctors_to_color[value]) for key, value in driver_to_ctors.items() ]
driver_to_color = dict(driver_to_color)


In [67]:
driver_and_team
team_to_driver = dict()
for key, value in driver_to_team.items():
    if value in team_to_driver.keys():
        team_to_driver[value].append(key)
    else:
        team_to_driver[value] = [key]

In [68]:
team_to_driver

{'Williams': ['ALB', 'SAR'],
 'Aston Martin': ['ALO', 'STR'],
 'Alfa Romeo': ['BOT', 'ZHO'],
 'AlphaTauri': ['DEV', 'LAW', 'RIC', 'TSU'],
 'Alpine F1 Team': ['GAS', 'OCO'],
 'Mercedes': ['HAM', 'RUS'],
 'Haas F1 Team': ['HUL', 'MAG'],
 'Ferrari': ['LEC', 'SAI'],
 'McLaren': ['NOR', 'PIA'],
 'Red Bull': ['PER', 'VER']}

In [58]:
grouped = df.groupby("driverCode")



[13, 10, 5, 5, 5, 5, 5, 11, 10, 10, 10, 10, 10, 10, 10, 10, 10]

In [100]:
series = []

for driver,value in grouped.groups.items():
    driver_dict = {
        "name":driver,
        "type": "line",
        "lineStyle": {"color": driver_to_color[driver]},
        "data" : grouped.get_group(driver)["positions"].tolist()
    }
    series.append(driver_dict)

series



[{'name': 'ALB',
  'type': 'line',
  'lineStyle': {'color': '#0073C2FF'},
  'data': [13, 10, 5, 5, 5, 5, 5, 11, 10, 10, 10, 10, 10, 10, 10, 10, 10]},
 {'name': 'ALO',
  'type': 'line',
  'lineStyle': {'color': '#EFC000FF'},
  'data': [20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   19,
   19,
   19]},
 {'name': 'BOT',
  'type': 'line',
  'lineStyle': {'color': '#868686FF'},
  'data': [15, 12, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9]},
 {'name': 'DEV',
  'type': 'line',
  'lineStyle': {'color': '#CD534CFF'},
  'data': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 {'name': 'GAS',
  'type': 'line',
  'lineStyle': {'color': '#7AA6DCFF'},
  'data': [14, 12, 9, 9, 13, 13, 13, 13, 12, 11, 11, 11, 13, 13, 13, 12, 12]},
 {'name': 'HAM',
  'type': 'line',
  'lineStyle': {'color': '#003C67FF'},
  'data': [18,
   18,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   20,
   20,
   20]},
 {'name': 'HU

In [129]:
import pandas as pd
ctors_to_color = pd.DataFrame({ "constructor":ctors.tolist(), "color" : colors})
df = df.merge(ctors_to_color,on='constructor',how='left')
plotdata = [df.columns.tolist()]
plotdata.extend(df.values.tolist())





In [130]:
plotdata


[['race',
  'driverCode',
  'points',
  'positions',
  'constructor',
  'color_x',
  'color_y'],
 [1, 'ALB', 1.0, 13, 'Williams', '#0073C2FF', '#0073C2FF'],
 [1, 'ALO', 15.0, 20, 'Aston Martin', '#EFC000FF', '#EFC000FF'],
 [1, 'BOT', 4.0, 15, 'Alfa Romeo', '#868686FF', '#868686FF'],
 [1, 'DEV', 0.0, 1, 'AlphaTauri', '#CD534CFF', '#CD534CFF'],
 [1, 'GAS', 2.0, 14, 'Alpine F1 Team', '#7AA6DCFF', '#7AA6DCFF'],
 [1, 'HAM', 10.0, 18, 'Mercedes', '#003C67FF', '#003C67FF'],
 [1, 'HUL', 0.0, 1, 'Haas F1 Team', '#8F7700FF', '#8F7700FF'],
 [1, 'LAW', 0.0, 1, 'AlphaTauri', '#CD534CFF', '#CD534CFF'],
 [1, 'LEC', 0.0, 1, 'Ferrari', '#3B3B3BFF', '#3B3B3BFF'],
 [1, 'MAG', 0.0, 1, 'Haas F1 Team', '#8F7700FF', '#8F7700FF'],
 [1, 'NOR', 0.0, 1, 'McLaren', '#A73030FF', '#A73030FF'],
 [1, 'OCO', 0.0, 1, 'Alpine F1 Team', '#7AA6DCFF', '#7AA6DCFF'],
 [1, 'PER', 18.0, 21, 'Red Bull', '#4A6990FF', '#4A6990FF'],
 [1, 'PIA', 0.0, 1, 'McLaren', '#A73030FF', '#A73030FF'],
 [1, 'RIC', 0.0, 1, 'AlphaTauri', '#CD534

In [113]:
ls

[['race', 'driverCode', 'points', 'positions', 'constructor'],
 [1, 'ALB', 1.0, 13, 'Williams'],
 [1, 'ALO', 15.0, 20, 'Aston Martin'],
 [1, 'BOT', 4.0, 15, 'Alfa Romeo'],
 [1, 'DEV', 0.0, 1, 'AlphaTauri'],
 [1, 'GAS', 2.0, 14, 'Alpine F1 Team'],
 [1, 'HAM', 10.0, 18, 'Mercedes'],
 [1, 'HUL', 0.0, 1, 'Haas F1 Team'],
 [1, 'LAW', 0.0, 1, 'AlphaTauri'],
 [1, 'LEC', 0.0, 1, 'Ferrari'],
 [1, 'MAG', 0.0, 1, 'Haas F1 Team'],
 [1, 'NOR', 0.0, 1, 'McLaren'],
 [1, 'OCO', 0.0, 1, 'Alpine F1 Team'],
 [1, 'PER', 18.0, 21, 'Red Bull'],
 [1, 'PIA', 0.0, 1, 'McLaren'],
 [1, 'RIC', 0.0, 1, 'AlphaTauri'],
 [1, 'RUS', 6.0, 16, 'Mercedes'],
 [1, 'SAI', 12.0, 19, 'Ferrari'],
 [1, 'SAR', 0.0, 1, 'Williams'],
 [1, 'STR', 8.0, 17, 'Aston Martin'],
 [1, 'TSU', 0.0, 1, 'AlphaTauri'],
 [1, 'VER', 25.0, 22, 'Red Bull'],
 [1, 'ZHO', 0.0, 1, 'Alfa Romeo'],
 [2, 'ALB', 1.0, 10, 'Williams'],
 [2, 'ALO', 30.0, 20, 'Aston Martin'],
 [2, 'BOT', 4.0, 12, 'Alfa Romeo'],
 [2, 'DEV', 0.0, 1, 'AlphaTauri'],
 [2, 'GAS', 4.0,

{'Williams': '#0073C2FF',
 'Aston Martin': '#EFC000FF',
 'Alfa Romeo': '#868686FF',
 'AlphaTauri': '#CD534CFF',
 'Alpine F1 Team': '#7AA6DCFF',
 'Mercedes': '#003C67FF',
 'Haas F1 Team': '#8F7700FF',
 'Ferrari': '#3B3B3BFF',
 'McLaren': '#A73030FF',
 'Red Bull': '#4A6990FF'}

In [166]:
import yukinator
import pandas as pd
import numpy as np
import datetime
import re

import requests



def get_wikipedia_main_pic_url(article_title):
    endpoint = "https://en.wikipedia.org/w/api.php"

    params = {
        "action": "query",
        "format": "json",
        "titles": article_title,
        "prop": "pageimages",
        "pithumbsize": 300,  # Adjust the thumbnail size as needed
    }

    response = requests.get(endpoint, params=params)
    data = response.json()

    page_id = list(data["query"]["pages"].keys())[0]
    main_image_url = data["query"]["pages"][page_id]["thumbnail"]["source"]

    return main_image_url






In [5]:

def get_driver_pic_url():
    YEAR = datetime.datetime.now().year
    y = yukinator.Yuki()
    current_drivers_standings = y.get_drivers_standings(year=YEAR)
    drivers = pd.DataFrame([driver.to_flat_dict() for driver in y.get_drivers(year=YEAR)])
    # Define the regex pattern to match the input text
    pattern = r"http://en.wikipedia.org/wiki/"
    # Define the replacement text with the modified URL
    replacement = r""
    drivers['article_title'] = drivers['url'].apply(lambda x: re.sub(pattern, replacement, x))

    conditions = [drivers["article_title"]=='Alexander_Albon', drivers["article_title"]=='Nico_H%C3%BClkenberg',drivers["article_title"]=='Sergio_P%C3%A9rez']
    values = ['Alex_Albon','Nico_Hülkenberg','Sergio_Pérez']

    drivers["article_title"] = np.select(conditions, values, default=drivers["article_title"])

    def try_get_wikipedia_main_pic_url(x):
        try:
            result = get_wikipedia_main_pic_url(x)
        except Exception as e:
            # Handle the exception (e.g., print an error message)
            print(f"Error processing {x}: {e}")
            result = None  

        return result
    # drivers
    
    
    return [ (driver_code, (try_get_wikipedia_main_pic_url(driver_article_title))) for driver_code, driver_article_title in dict(zip(drivers["code"],drivers["article_title"])).items()]


driver_pic_url = get_driver_pic_url()



NameError: name 'datetime' is not defined

In [208]:
YEAR = datetime.datetime.now().year
y = yukinator.Yuki()
current_drivers_standings = y.get_drivers_standings(year=YEAR)
drivers = pd.DataFrame([driver.to_flat_dict() for driver in y.get_drivers(year=YEAR)])
# Define the regex pattern to match the input text
pattern = r"http://en.wikipedia.org/wiki/"
# Define the replacement text with the modified URL
replacement = r""
drivers['article_title'] = drivers['url'].apply(lambda x: re.sub(pattern, replacement, x))

conditions = [drivers["article_title"]=='Alexander_Albon', drivers["article_title"]=='Nico_H%C3%BClkenberg',drivers["article_title"]=='Sergio_P%C3%A9rez']
values = ['Alex_Albon','Nico_Hülkenberg','Sergio_Pérez']

drivers["article_title"] = np.select(conditions, values, default=drivers["article_title"])

In [215]:
dict

{'name': 'ZHO',
 'type': 'line',
 'lineStyle': {'color': '#868686FF'},
 'data': [1, 1, 8, 7, 6, 6, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]}

In [143]:
import re
# Input text
text = "http://en.wikipedia.org/wiki/Alexander_Albon"

# Define the regex pattern to match the input text
pattern = r"http://en.wikipedia.org/wiki/(\w+)"

# Define the replacement text with the modified URL
replacement = r"https://en.wikipedia.org/w/api.php/\1"

# Use re.sub() to replace the matched pattern with the replacement
modified_text = re.sub(pattern, replacement, text)

# Print the modified text
print(modified_text)

https://en.wikipedia.org/w/api.php/Alexander_Albon


In [170]:
import requests




get_wikipedia_main_pic_url('Fernando_Alonso')


'https://upload.wikimedia.org/wikipedia/commons/thumb/2/2b/Alonso_2016.jpg/300px-Alonso_2016.jpg'

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [136]:
import requests

# Specify the Wikipedia article title
article_title = "Albert_Einstein"

# Define the API endpoint for Wikipedia
endpoint = "https://en.wikipedia.org/w/api.php"

# Parameters for the API request to fetch the thumbnail info
params = {
    "action": "query",
    "format": "json",
    "titles": article_title,
    "prop": "pageimages",
    "pithumbsize": 300,  # Adjust the thumbnail size as needed
}

# Send the API request
response = requests.get(endpoint, params=params)
data = response.json()

# Extract and display the thumbnail info
page_id = list(data["query"]["pages"].keys())[0]
thumbnail_info = data["query"]["pages"][page_id]["thumbnail"]

if "source" in thumbnail_info:
    print("Thumbnail URL:", thumbnail_info["source"])
    print("Thumbnail Width:", thumbnail_info["width"])
    print("Thumbnail Height:", thumbnail_info["height"])
else:
    print("No thumbnail available for this article.")

# You can also use the thumbnail URL to display the image in your application.


Thumbnail URL: https://upload.wikimedia.org/wikipedia/commons/thumb/3/3e/Einstein_1921_by_F_Schmutzer_-_restoration.jpg/300px-Einstein_1921_by_F_Schmutzer_-_restoration.jpg
Thumbnail Width: 300
Thumbnail Height: 394


In [75]:
def get_team_to_driver(df):
    driver_to_team = (
        df[["driverCode", "constructor"]]
        .drop_duplicates()
        .set_index("driverCode")
        .to_dict(orient="index")
    )
    team_to_driver = dict()
    for key in driver_to_team.keys():
        value = driver_to_team[key]['constructor']
        if value in team_to_driver.keys():
            team_to_driver[value].append(key)
        else:
            team_to_driver[value] = [key]
    return team_to_driver

In [77]:

current_drivers = (
    df[df["race"] == df["race"].max()]
    .set_index("driverCode")
    .to_dict(orient="index")
)
# drivers = driver_info.set_index("code")
# for key in current_drivers.keys():
#     drivers[key].update(current_drivers[key])





In [78]:
current_drivers

{'ALB': {'race': 17,
  'points': 23.0,
  'positions': 13,
  'constructor': 'Williams'},
 'ALO': {'race': 17,
  'points': 183.0,
  'positions': 4,
  'constructor': 'Aston Martin'},
 'BOT': {'race': 17,
  'points': 10.0,
  'positions': 14,
  'constructor': 'Alfa Romeo'},
 'DEV': {'race': 17,
  'points': 0.0,
  'positions': 20,
  'constructor': 'AlphaTauri'},
 'GAS': {'race': 17,
  'points': 46.0,
  'positions': 11,
  'constructor': 'Alpine F1 Team'},
 'HAM': {'race': 17,
  'points': 194.0,
  'positions': 3,
  'constructor': 'Mercedes'},
 'HUL': {'race': 17,
  'points': 9.0,
  'positions': 15,
  'constructor': 'Haas F1 Team'},
 'LAW': {'race': 17,
  'points': 2.0,
  'positions': 19,
  'constructor': 'AlphaTauri'},
 'LEC': {'race': 17,
  'points': 145.0,
  'positions': 6,
  'constructor': 'Ferrari'},
 'MAG': {'race': 17,
  'points': 3.0,
  'positions': 17,
  'constructor': 'Haas F1 Team'},
 'NOR': {'race': 17,
  'points': 136.0,
  'positions': 7,
  'constructor': 'McLaren'},
 'OCO': {'race

In [79]:
constructors = pd.DataFrame(
        [driver.to_flat_dict() for driver in y.get_constructors(year=YEAR)]
    )



In [84]:
constructors['name']

0        Alfa Romeo
1        AlphaTauri
2    Alpine F1 Team
3      Aston Martin
4           Ferrari
5      Haas F1 Team
6           McLaren
7          Mercedes
8          Red Bull
9          Williams
Name: name, dtype: object

In [87]:
constructors['url']

0    http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...
1     http://en.wikipedia.org/wiki/Scuderia_AlphaTauri
2          http://en.wikipedia.org/wiki/Alpine_F1_Team
3    http://en.wikipedia.org/wiki/Aston_Martin_in_F...
4        http://en.wikipedia.org/wiki/Scuderia_Ferrari
5            http://en.wikipedia.org/wiki/Haas_F1_Team
6                 http://en.wikipedia.org/wiki/McLaren
7    http://en.wikipedia.org/wiki/Mercedes-Benz_in_...
8         http://en.wikipedia.org/wiki/Red_Bull_Racing
9    http://en.wikipedia.org/wiki/Williams_Grand_Pr...
Name: url, dtype: object

In [88]:
def get_complete_driver_info(df, driver_info):
    """get favorite driver"""
    current_drivers = (
        df[df["race"] == df["race"].max()]
        .set_index("driverCode")
        .to_dict(orient="index")
    )
    drivers = driver_info.set_index("code").to_dict(orient="index")
    for key in current_drivers.keys():
        drivers[key].update(current_drivers[key])
    return drivers


In [90]:
 complete_driver_info = get_complete_driver_info(df,drivers)

In [99]:
complete_driver_info

profile_items = ['givenName','familyName','dateOfBirth','nationality','permanentNumber','contructor','points','positions', "constructo"]
profile = dict()

for key1,value1 in complete_driver_info.items():
    profile[key1] = dict()
    # temp = {"profile": {key2: value2} for key2, value2 in value1.items() if key2 in profile_items }
    [(key2,value2) for key2, value2 in value1.items() if key2 in profile_items ]
    profile[key1]['profile'] = dict([(key2,value2) for key2, value2 in value1.items() if key2 in profile_items ])
    for key2, value2 in value1.items():
        if key2 not in profile_items:
            profile[key1][key2] = value2




{'profile': {'positions': 13}}
{'profile': {'positions': 4}}
{'profile': {'positions': 14}}
{'profile': {'positions': 20}}
{'profile': {'positions': 11}}
{'profile': {'positions': 3}}
{'profile': {'positions': 15}}
{'profile': {'positions': 19}}
{'profile': {'positions': 6}}
{'profile': {'positions': 17}}
{'profile': {'positions': 7}}
{'profile': {'positions': 12}}
{'profile': {'positions': 2}}
{'profile': {'positions': 9}}
{'profile': {'positions': 20}}
{'profile': {'positions': 8}}
{'profile': {'positions': 5}}
{'profile': {'positions': 20}}
{'profile': {'positions': 10}}
{'profile': {'positions': 17}}
{'profile': {'positions': 1}}
{'profile': {'positions': 16}}


In [92]:
original_dict = {
    'driver1': {'url': 'url1', 'givenName': 'name1', 'race': 'race1'},
    'driver2': {'url': 'url2', 'givenName': 'name2', 'race': 'race2'}
}

# Create a new dictionary with the "urc" key and third-level dictionaries
new_dict = {key: {'urc': value} for key, value in original_dict.items()}

# Print the resulting dictionary
print(new_dict)

{'driver1': {'urc': {'url': 'url1', 'givenName': 'name1', 'race': 'race1'}}, 'driver2': {'urc': {'url': 'url2', 'givenName': 'name2', 'race': 'race2'}}}


In [106]:
key1,value1 = complete_driver_info.items()

KeyError: 1

In [116]:
key1, value1 = list(complete_driver_info.items())[0]

In [115]:
value1

{'driverId': 'alonso',
 'url': 'http://en.wikipedia.org/wiki/Fernando_Alonso',
 'givenName': 'Fernando',
 'familyName': 'Alonso',
 'dateOfBirth': datetime.date(1981, 7, 29),
 'nationality': 'Spanish',
 'permanentNumber': 14,
 'article_title': 'Fernando_Alonso',
 'pic_url': 'https://upload.wikimedia.org/wikipedia/commons/thumb/2/2b/Alonso_2016.jpg/300px-Alonso_2016.jpg',
 'contructor': 'Aston Martin',
 'race': 17,
 'points': 183.0,
 'positions': 4,
 'constructor': 'Aston Martin'}

In [137]:
profile

{'ALB': {'profile': {'givenName': 'Alexander',
   'familyName': 'Albon',
   'dateOfBirth': datetime.date(1996, 3, 23),
   'nationality': 'Thai',
   'permanentNumber': 23,
   'contructor': 'Williams',
   'points': 23.0,
   'positions': 13},
  'driverId': 'albon',
  'url': 'http://en.wikipedia.org/wiki/Alexander_Albon',
  'article_title': 'Alex_Albon',
  'pic_url': 'https://upload.wikimedia.org/wikipedia/commons/thumb/1/1f/Alex_albon_%2851383514844%29_%28cropped%29.jpg/300px-Alex_albon_%2851383514844%29_%28cropped%29.jpg',
  'race': 17,
  'constructor': 'Williams'}}